In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
from glob import glob
import os
from sklearn.model_selection import train_test_split
from PIL import Image as im




In [3]:
from datasets import load_dataset         

c:\Users\vivek\anaconda3\envs\TF\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("mattmdjaga/human_parsing_dataset")


In [5]:
img_list=[]
mask_list=[]
for i in range(10000):
    img_list.append(dataset["train"][i]["image"])
    mask_list.append(dataset["train"][i]["mask"])


In [140]:
# for i in range(10000):
#     mask_list[i].save(f"dataset\\train\\Categories\\img_{i}.png")
l=os.listdir("dataset\\train\\Categories")

# creating rgb mask

In [143]:
l.sort()


10000

# converting mask to rgb

In [45]:
# def grayscale_to_rgb_mask(image,classes,rgb_color_map):
#     image=np.expand_dims(image,axis=-1)
#     h, w, _=image.shape
#     image=image.astype(np.int32)

#     output=[]
#     for i,pixcel in enumerate(image.flatten()):
#         output.append(rgb_color_map[pixcel])
#     output=np.reshape(output, (h,w,3))
#     return output

In [46]:
# a1=np.array(mask_list[0])
# a1=np.expand_dims(a1,axis=-1)
# print(a1.shape)

(600, 400, 1)


In [52]:
# for i in range(5000,10000):
#     output=grayscale_to_rgb_mask(np.array(mask_list[i]),classes, rgb_color_map )
#     cv2.imwrite(f"img_{i}.jpg",output)

In [9]:
def create_path(path):
    if not os.path.exists(path):
        os.makedirs(path)

def get_array_mask(num,arr):
    mask_array=arr[240000*num:(240000*num)+240000].reshape(600,400)
    return mask_array


In [10]:
# #saving the mask_list in np array 
# save_array=np.array(mask_list[0]).flatten()
# for i in range(1,500):
#     save_array=np.append(save_array,np.array(mask_list[i]).flatten())
#     print(i)
# np.save("mask_array",save_array)


# global data

In [11]:
label_map={"0": "Background", "1": "Hat", "2": "Hair", "3": "Sunglasses", "4": "Upper-clothes", "5": "Skirt", "6": "Pants", "7": "Dress", "8": "Belt", "9": "Left-shoe", "10": "Right-shoe", "11": "Face", "12": "Left-leg", "13": "Right-leg", "14": "Left-arm", "15": "Right-arm", "16": "Bag", "17": "Scarf"}
rgb_color_map=[[0,0,0], [0,0,128], [0,0,255], [0,85,0], [51,0,170], [0,85,255], [85,0,0], [221,119,0], [0,85,85], [85,85,0], [0,51,85], [128,86,52], [0,28,0], [255,0,0], [221,170,51], [225,225,0], [170,255,85], [85,255,170]]
classes=["Background",  "Hat",  "Hair",  "Sunglasses",  "Upper-clothes", "Skirt", "Pants",  "Dress", "Belt",  "Left-shoe", "Right-shoe", "Face", "Left-leg", "Right-leg", "Left-arm", "Right-arm", "Bag", "Scarf"]

COLOR_MAP=[k for k in range(18)]

create_path("dataset\\train\\input")
create_path("dataset\\train\\Categories")


18


# data preprocessing

# 1)hyper parameter

In [12]:
IMAGE_HEIGHT=600
IMAGE_WIDTH=400
NUM_CLASSES=18
INPUT_SHAPE=(IMAGE_HEIGHT,IMAGE_WIDTH,3)
BATCH_SIZE=8
NUM_EPOCH=100
LR=1e-4

#dataset paths
dataset_path="dataset\\train"
model_path=os.path.join("files","model.h5")
csv_path=os.path.join("files","data.csv")


# loading the dataset

In [114]:

def load_dataset(path,split=0.2):
    #loading the images
    train_x=sorted(glob(os.path.join(path, "input", "*")))
    train_y=sorted(glob(os.path.join(path, "Categories", "*")))

    split_size = int(split * len(train_x))

    train_x, valid_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(train_y, test_size=split_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y) 


(train_x,train_y), (test_x, test_y), (valid_x, valid_y)=load_dataset(dataset_path)


# color map processing

In [14]:
def get_colormap():
    colormap=[k for k in range(18)]
    classmap=[
        "Background",
        "Hat",
        "Hair",
        "Sunglasses",
        "Upper-clothes",
        "Skirt",
        "Pants",
        "Dress",
        "Belt",
        "Left-shoe",
        "Right-shoe",
        "Face",
        "Left-leg",
        "Right-leg",
        "Left-arm",
        "Right-arm",
        "Bag",
        "Scarf"
    ]
    return classmap, colormap

CLASSES, COLORMAP=get_colormap()

# dataset pipeline

In [119]:
def read_img(path):

    x = cv2.imread(path, cv2.IMREAD_COLOR)
    cv2.imwrite("testimg.jpg",x)
    x = x / 255.0
    x = x.astype(np.float32)
    return x
    


In [120]:
def read_mask(path):
    x=cv2.imread(path,cv2.IMREAD_ANYDEPTH)
    output=[]
    for color in COLORMAP:
        cmap=np.equal(x, color)
        output.append(cmap)
    output=np.stack(output, axis=-1)
    output=output.astype(np.uint8)
    return output

dataset\train\input\img_8728.jpg


In [147]:
def preprocess(x,y):
    """x and y represent image path and mask path"""
    def f(x,y):
        x=x.decode()
        y=y.decode()
        x=read_img(x)
        y=read_mask(y)
        return x,y
    image,mask=tf.numpy_function(f, [x, y],[tf.float32,tf.uint8])
    image.set_shape([IMAGE_HEIGHT,IMAGE_WIDTH,3 ])
    mask.set_shape([IMAGE_HEIGHT,IMAGE_WIDTH,NUM_CLASSES])
    return image,mask

                

In [148]:
def tf_dataset(x,y,batch=BATCH_SIZE):
    dataset=tf.data.Dataset.from_tensor_slices((x,y))
    dataset=dataset.shuffle(buffer_size=5000)
    dataset=dataset.map(preprocess)
    dataset=dataset.batch(batch)
    dataset=dataset.prefetch(2)
    return dataset

In [149]:
train_dataset=tf_dataset(train_x,train_y,batch=BATCH_SIZE)
valid_dataset=tf_dataset(valid_x,valid_y,batch=BATCH_SIZE)


In [150]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

if __name__ == "__main__":
    input_shape = (384, 384, 3)
    model = build_unet(input_shape,18)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(LR)
    )

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=NUM_EPOCH,
        callbacks=callbacks
    )
    # model.summary()

ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[3,3,1024,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

# model

In [133]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Input, concatenate
from tensorflow.keras.models import Model

def build_vgg16_segmentation_model(input_shape=(384, 384, 3), num_classes=21):
    # Load the VGG16 model without the top fully connected layers
    vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Convert the fully connected layers into convolutional layers
    x = Conv2D(4096, (7, 7), activation='relu', padding='same', name='fc1')(vgg16.output)
    x = Conv2D(4096, (1, 1), activation='relu', padding='same', name='fc2')(x)
    x = Conv2D(num_classes, (1, 1), activation='linear', padding='same', name='predictions')(x)

    # Upsample the feature maps to match the original image size
    x = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same', name='up1')(x)
    x = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same', name='up2')(x)
    x = Conv2DTranspose(num_classes, kernel_size=(16, 16), strides=(8, 8), padding='same', name='up3')(x)

    # Add skip connections to fuse features from different layers
    f1 = vgg16.get_layer('block1_conv2').output
    f2 = vgg16.get_layer('block2_conv2').output
    f3 = vgg16.get_layer('block3_conv3').output
    f4 = vgg16.get_layer('block4_conv3').output
    f5 = vgg16.get_layer('block5_conv3').output

    # Upsample feature maps from each block to match the final output size
    f2 = Conv2DTranspose(num_classes, kernel_size=(4, 4), strides=(2, 2), padding='same', name='up_f2')(f2)
    f3 = Conv2DTranspose(num_classes, kernel_size=(8, 8), strides=(4, 4), padding='same', name='up_f3')(f3)
    f4 = Conv2DTranspose(num_classes, kernel_size=(16, 16), strides=(8, 8), padding='same', name='up_f4')(f4)
    f5 = Conv2DTranspose(num_classes, kernel_size=(32, 32), strides=(16, 16), padding='same', name='up_f5')(f5)

    # Concatenate the upsampled feature maps
    x = concatenate([x, f1, f2, f3, f4, f5], axis=3)

    # Final upsampling to match the original image size
    x = Conv2DTranspose(num_classes, kernel_size=(32, 32), strides=(16, 16), padding='same', name='up4')(x)

    # Create the model
    model = Model(inputs=vgg16.input, outputs=x)

    return model

# Define the model
input_shape = (384, 384, 3)
num_classes = 18  # Set the number of classes for segmentation
model = build_vgg16_segmentation_model(input_shape=input_shape, num_classes=num_classes)

# Print the model summary
model.summary()


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} OOM when allocating tensor with shape[7,7,512,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:StatelessRandomUniformV2]

In [36]:
# def read_mask(path):
    
#     #use the image number to generate corropoding cmap
#     x=np.load("arrfile.npy")
#     #coverting to jpg image for prcessingW
#     output=[]
#     for i,color in enumerate(COLORMAP):
#         cmap=np.equal(x, color)
#         cv2.imwrite(f"test_map_{i}.jpg", cmap*255)
        
    

In [117]:
#converts a given mask image into binary mask for all classes and save it

def readMask(x):
    #x is an np array not a mask_list object


    """Mask processing"""
    output=[]
    for  i, color in enumerate(COLOR_MAP):
        color=np.array(color)
        cmap=np.equal(x,color)
        cv2.imwrite(f"cmap_from_array_{i}.png",cmap*255)
        
        
    #     output.append(cmap)

    # output=np.stack(output, axis=-1)
    # output=output.astype(np.uint8)
    # """the output file has all the masked prcessed from class 1 to 17"""
    # return output

In [ ]:
# sfasf

In [18]:
img1=np.array(np.array(img_list[0]))

In [23]:
from PIL import Image

In [27]:
img_list[0].save("out.jpg","jpeg")

In [25]:
im=Image.open("img1.png")
print(type(im))
# im.show()

<class 'PIL.PngImagePlugin.PngImageFile'>


# image conversion process

# Image processing- fetching data using api and saving it localy

In [64]:
def build_image_mask_dataset(img_list,mask_list):    
    def save_img(img_list):
        for k in range(1000):
            img_list[k].save(f"D:\\work\\reserch work\\shivgami mam\\dataset\\train\\input\\img_{k}.jpg","jpeg")

    def save_mask(mask_list):
        for k in range(1000):
            mask_list[k].save(f"D:\\work\\reserch work\\shivgami mam\\dataset\\train\\Categories\\img_{k}.jpg","jpeg")

    save_img(img_list)
    save_mask(mask_list)         

build_image_mask_dataset(img_list,mask_list)      

# image conversion process - end

In [139]:
def preprocess(x,y):
    """
    x-represent the image
    y-represent the mask
    """
    def get_image_mask(x,y):
        return readImg(x), readMask(y)

    image,mask=tf.numpy_function(get_image_mask, [x,y], [tf.float32, tf.uint8])
    image.set_shape([600,400],3)
    mask.set_shape([600,400,18])
    """600*400 image size and 18 is the number of output classes"""
    return image,mask

In [96]:
preprocess(np.array(img_list[0]),np.array(mask_list[0]))

TypeError: Can't convert object to 'str' for 'filename'

In [86]:
def tf_dataset(x, y, batch=8):
    dataset=tf.data.Dataset.from_tensor_slices((x,y))
    dataset=dataset.shuffle(buffer_size=5000)
    dataset=dataset.map(preprocess)
    dataset=dataset.batch(batch)
    dataset=dataset.prefetch(2)
    return dataset

In [94]:
"""dataset pipeline"""
train_dataset=tf_dataset(train_x,train_y,batch=4)
test_dataset=tf_dataset(test_x,test_y,batch=4)

ValueError: Attempt to convert a value (<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=400x600 at 0x195FF551460>) with an unsupported type (<class 'PIL.JpegImagePlugin.JpegImageFile'>) to a Tensor.

In [82]:
tempimg=np.array(mask_list[0],dtype=np.float32)
readMask(tempimg)

In [57]:
b=np.array(1)

In [58]:
np.equal(tempimg,b)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

numpy.ndarray

0
2
11
7
15
14
8
16
13
12
9
10


# trash code


In [53]:
def build_fcn_vgg16(input_shape):
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    
    # Taking only the convolutional part of VGG16
    conv_layers = vgg16.get_layer('block5_conv3').output
    
    # Adding fully convolutional layers
    conv6 = Conv2D(4096, (7, 7), activation='relu', padding='same', name='conv6')(conv_layers)
    conv7 = Conv2D(4096, (1, 1), activation='relu', padding='same', name='conv7')(conv6)
    conv8 = Conv2D(18, (1, 1), activation='softmax', padding='same', name='conv8')(conv7)
    
    # Upsampling
    upsampled = UpSampling2D(size=(32, 32), interpolation='bilinear')(conv8)
    
    model = Model(inputs=vgg16.input, outputs=upsampled)
    return model

In [54]:
def build_phpc(input_shape):
    # Image-level parsing network
    image_level_model = build_fcn_vgg16(input_shape)
    
    # Head-parsing sub-network
    head_input = tf.keras.layers.Input(shape=(input_shape[0]*2, input_shape[1]*2, input_shape[2]))
    head_model = build_fcn_vgg16((input_shape[0]*2, input_shape[1]*2, input_shape[2]))
    
    # Body-parsing sub-network
    body_input = tf.keras.layers.Input(shape=(input_shape[0]*2, input_shape[1]*2, input_shape[2]))
    body_model = build_fcn_vgg16((input_shape[0]*2, input_shape[1]*2, input_shape[2]))
    
    # Combining all the networks
    combined = Concatenate()([image_level_model.output, head_model.output, body_model.output])
    final_output = Conv2D(18, (1, 1), activation='softmax', padding='same')(combined)
    
    model = Model(inputs=[image_level_model.input, head_input, body_input], outputs=final_output)
    return model


In [55]:

input_shape = (384, 384, 3)
model = build_phpc(input_shape)
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# model.summary()


ResourceExhaustedError: {{function_node __wrapped__Mul_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:Mul]

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(input, num_filters):
    x = conv_block(input, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs, name="U-Net")
    return model

if __name__ == "__main__":
    input_shape = (384, 384, 3)
    model = build_unet(input_shape,18)
    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(LR)
    )

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path, append=True),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(train_dataset,
        validation_data=valid_dataset,
        epochs=NUM_EPOCH,
        callbacks=callbacks
    )
    # model.summary()